In [2]:
import pandas as pd
from ast import literal_eval
from langchain.evaluation import load_evaluator
import numpy as np
import sys

sys.path.append("../")
from eval.evaluation import evaluate_string_similarity, evaluate_number_similarity

distance_evaluator = load_evaluator("string_distance")


In [3]:
from langchain.evaluation import StringDistance
distance_evaluator = load_evaluator("string_distance", distance=StringDistance.LEVENSHTEIN)

In [4]:
gt = pd.read_csv("/home/ec2-user/project/src/eval/ground_truth/contracts_v2.csv")
df = pd.read_csv("output/performance_test.csv")
df

,Unnamed: 0,contract_filename,elapsed_time,response
0,0,ArbeitsvertragZÄAssistentin.pdf,29.475872,"{""start_date"":[""15.02.2021""],""sign_date"":[""15...."
1,1,140402 AV.pdf,22.566979,"{""start_date"":[""01.08.2014""],""sign_date"":[""02...."
2,2,160729_Dienstvertrag_HB.pdf,74.468362,"{""start_date"":[""N/A""],""sign_date"":[""29.07.2016..."
3,3,Arbeitsvertrag Bain.pdf,27.169661,"{""start_date"":[""01.03.2016""],""sign_date"":[""07...."
4,4,Arbeitsvertrag Oper Leipzig.pdf,17.845933,"{""start_date"":[""01.04.2022""],""sign_date"":[""N/A..."
5,5,Arbeitsvertrag TUM.pdf,14.696686,"{""start_date"":[""01.03.2018""],""sign_date"":[""21...."
6,6,ArbeitsvertragCheck24.pdf,54.533255,"{""start_date"":[""01.09.2019""],""sign_date"":[""01...."
7,7,AV_2.pdf,39.903615,"{""start_date"":[""24.03.2015""],""sign_date"":[""24...."
8,8,AV_GF.pdf,45.133483,"{""start_date"":[""01.01.2015""],""sign_date"":[""201..."
9,9,AV123.pdf,42.221631,"{""start_date"":[""30.08.2011""],""sign_date"":[""30...."


In [5]:
new_columns = list(literal_eval(df.iloc[0].response).keys())
df[new_columns] = df.response.apply(literal_eval).apply(pd.Series)
for col in new_columns:
    df[col] = df[col].apply(lambda x: x[0])

In [6]:
gt["notice_period"] = gt["notice_period (M)"].apply(lambda x: x.split(" ")[0]).astype(float, errors="ignore")
gt.notice_period.replace("Not", "Not found", inplace=True)

gt.fillna("Not found", inplace=True)

/tmp/ipykernel_14945/3680041924.py:4: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'Not found' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  gt.fillna("Not found", inplace=True)


In [7]:
df.sort_values(by="contract_filename", ascending=True, inplace=True)
gt.sort_values(by="contract_filename", ascending=True, inplace=True)

# Overall results

In [8]:
skip_cols = ["address_employee", "notice_period", "annual_gross_salary"]

selected_cols = list(set(new_columns) - set(skip_cols))

res = df[["contract_filename"] + selected_cols].merge(gt[["contract_filename"] + selected_cols], on="contract_filename", how="outer", suffixes=("_pred", "_gt"))

In [9]:
for col in selected_cols:
    res[col+"_xscore"] = res.apply(lambda x: evaluate_string_similarity(x[col+"_gt"], x[col+"_pred"], distance_evaluator, 0.2) if x[col+"_gt"] != "Not found" else None, axis=1)
    
res = res[["contract_filename"] + sorted(res.columns.tolist()[1:])]

In [10]:
df.columns

Index(['Unnamed: 0', 'contract_filename', 'elapsed_time', 'response',
       'start_date', 'sign_date', 'employer_name', 'notice_period',
       'address_employer', 'address_employee', 'birth_date', 'job_title',
       'type_of_contract', 'annual_gross_salary'],
      dtype='object')

In [16]:
feat = "type_of_contract"
col_list = ["contract_filename", feat+"_gt", feat+"_pred", feat+"_xscore"]
res[col_list]

,contract_filename,type_of_contract_gt,type_of_contract_pred,type_of_contract_xscore
0,140402 AV.pdf,permanent,permanent,True
1,160729_Dienstvertrag_HB.pdf,permanent,permanent,True
2,AV123.pdf,permanent,permanent,True
3,AV_2.pdf,permanent,permanent,True
4,AV_GF.pdf,permanent,permanent,True
5,Arbeitsvertrag Bain.pdf,permanent,permanent,True
6,Arbeitsvertrag Oper Leipzig.pdf,temporary,fixed-term,False
7,Arbeitsvertrag TUM.pdf,temporary,fixed-term,False
8,ArbeitsvertragCheck24.pdf,permanent,permanent,True
9,ArbeitsvertragZÄAssistentin.pdf,temporary,fixed-term,False


In [48]:
14/22

0.6363636363636364

In [12]:
ratios = {}
for col in res.columns:
    if "xscore" in col:
        ratios[col] = res[col].mean()
        
ratios

{'address_employer_xscore': 0.6363636363636364,
 'birth_date_xscore': 1.0,
 'employer_name_xscore': 0.76,
 'job_title_xscore': 0.7619047619047619,
 'sign_date_xscore': 0.6086956521739131,
 'start_date_xscore': 0.782608695652174,
 'type_of_contract_xscore': 0.5652173913043478}

# Elapsed time

In [17]:
df.elapsed_time.describe()

count    25.000000
mean     27.169177
std      15.555648
min      11.416593
25%      14.863090
50%      22.566979
75%      35.320105
max      74.468362
Name: elapsed_time, dtype: float64

In [19]:
df[["contract_filename", "elapsed_time"]]

,contract_filename,elapsed_time
1,140402 AV.pdf,22.566979
2,160729_Dienstvertrag_HB.pdf,74.468362
9,AV123.pdf,42.221631
7,AV_2.pdf,39.903615
8,AV_GF.pdf,45.133483
3,Arbeitsvertrag Bain.pdf,27.169661
4,Arbeitsvertrag Oper Leipzig.pdf,17.845933
5,Arbeitsvertrag TUM.pdf,14.696686
6,ArbeitsvertragCheck24.pdf,54.533255
0,ArbeitsvertragZÄAssistentin.pdf,29.475872


In [15]:
df.columns

Index(['Unnamed: 0', 'contract_filename', 'elapsed_time', 'response',
       'start_date', 'sign_date', 'employer_name', 'notice_period',
       'address_employer', 'address_employee', 'birth_date', 'job_title',
       'type_of_contract', 'annual_gross_salary'],
      dtype='object')

In [14]:
df[["contract_filename", "notice_period", "annual_gross_salary"]]

KeyError: "None of [Index([('contract_filename', 'elapsed_time', 'address_employer_xscore', 'birth_date_xscore', 'employer_name_xscore', 'job_title_xscore', 'sign_date_xscore', 'start_date_xscore', 'type_of_contract_xscore')], dtype='object')] are in the [columns]"